Compering nested matched
Comparing Nested Mixed-Effects Models on Matched Cohort
This function performs likelihood ratio tests to compare nested mixed-effects logistic regression models (glmer) on a matched dataset created via propensity score matching.

Input: create_matched_cohort – a matched dataset.

Method:

Model 1 (reduced): includes the main exposure (LL_COVID_diagnosis...), age, race, and a random effect for provider (data_partner_id).

Model 2 (full): adds clinical and behavioral covariates (e.g., obesity, smoking, diabetes).

Test: Likelihood ratio test (anova) compares model fit.

Outcomes analyzed:

Postpartum depression

C-section

(Pre)eclampsia

Output: A table with chi-squared statistics, degrees of freedom, and p-values indicating whether the full model provides a significantly better fit.

In [ ]:


library(dplyr)
library(lme4)
# Compare the nested models
# Note that because the interactions are not significant, and we don't report models with interaction, we don't need to compare models with interaction terms

compare_nested_models_matched <- function(create_matched_cohort) {

    df <- create_matched_cohort

    # Function to extract outputs of Likelihood ratio tests
    compare_glmer_models_row <- function(data, outcome_var, dv_label = NULL) {
  if (is.null(dv_label)) dv_label <- outcome_var

  # Model 1 (reduced)
  fixed_mod1 <- paste0(
    outcome_var, " ~ LL_COVID_diagnosis_1y_during_1y_prior_to_1st_delivery_indicator + age + race_White + (1 | data_partner_id)"
  )

  # Model 2 (full)
  fixed_mod2 <- paste0(
    outcome_var, " ~ LL_COVID_diagnosis_1y_during_1y_prior_to_1st_delivery_indicator + age + race_White + ",
    "total_number_of_COVID_vaccine_doses + OBESITY_1y_during_1y_prior_to_1st_delivery_indicator + ",
    "hypertension_1y_during_1y_prior_to_1st_delivery_indicator + chronic_lung_disease_1y_during_1y_prior_to_1st_delivery_indicator + ",
    "tobacco_smoker_1y_during_1y_prior_to_1st_delivery_indicator + diabetes_combined_1y_during_1y_prior_to_1st_delivery_indicator + ",
    "total_visits + (1 | data_partner_id)"
  )

  # Fit models
  mod1 <- glmer(as.formula(fixed_mod1), data = data, family = binomial)
  mod2 <- glmer(as.formula(fixed_mod2), data = data, family = binomial)

  # Likelihood ratio test
  anova_result <- anova(mod1, mod2, test = "Chisq")
  df_diff <- anova_result$Df[2] - anova_result$Df[1]
  chi_sq <- round(anova_result$Chisq[2], 2)
  p_val <- anova_result$`Pr(>Chisq)`[2]
  p_val_str <- ifelse(p_val < 0.001, "< 0.001", sprintf("%.4f", p_val))

  # Return one row
  return(data.frame(
    dv_label = dv_label,
    chi_sq = chi_sq,
    df_diff = df_diff,
    p_val = p_val,
    p_val_str = p_val_str,
    stringsAsFactors = FALSE
  ))
}
    # Compare 2 models of postpartum depression
    output <- compare_glmer_models_row(data = df, outcome_var = "postpartum_depression", dv_label = "postpartum depression") %>%
    bind_rows(
        compare_glmer_models_row(data = df, outcome_var = "postpartum_csection", dv_label = "C-section")
    ) %>%
    bind_rows(
        compare_glmer_models_row(data = df, outcome_var = "pre_eclampsia_combined_1y_during_1y_prior_to_1st_delivery_indicator", dv_label = "(Pre)eclampsia")
    )

    return(output)

}